In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Scrapping Players Link

In [2]:
r = requests.get('https://sofifa.com/')

In [3]:
soup = BeautifulSoup(r.content, 'html.parser')

In [4]:
%%time
links = []
for i in range(0,20000,60):
    
    
    r = requests.get(f'https://sofifa.com/?offset={i}')
    soup = BeautifulSoup(r.content, 'html.parser')
    
    if i%100 == 0:
        print(f"Offset={i}")
    
    for j in range(0,120,2):
        link = soup.select('tr td.col-name')[j].select('a')[0].get('href')
        links.append(link)
    
    

Offset=0
Offset=300
Offset=600
Offset=900
Offset=1200
Offset=1500
Offset=1800
Offset=2100
Offset=2400
Offset=2700
Offset=3000
Offset=3300
Offset=3600
Offset=3900
Offset=4200
Offset=4500
Offset=4800
Offset=5100
Offset=5400
Offset=5700
Offset=6000
Offset=6300
Offset=6600
Offset=6900
Offset=7200
Offset=7500
Offset=7800
Offset=8100
Offset=8400
Offset=8700
Offset=9000
Offset=9300
Offset=9600
Offset=9900
Offset=10200
Offset=10500
Offset=10800
Offset=11100
Offset=11400
Offset=11700
Offset=12000
Offset=12300
Offset=12600
Offset=12900
Offset=13200
Offset=13500
Offset=13800
Offset=14100
Offset=14400
Offset=14700
Offset=15000
Offset=15300
Offset=15600
Offset=15900
Offset=16200
Offset=16500
Offset=16800
Offset=17100
Offset=17400
Offset=17700
Offset=18000
Offset=18300
Offset=18600
Offset=18900
Offset=19200
Offset=19500
Offset=19800
CPU times: user 7min 51s, sys: 919 ms, total: 7min 52s
Wall time: 9min 7s


In [5]:
len(links)

20040

In [6]:
links = list(set(links))

In [7]:
links = ['https://sofifa.com'+i for i in links]
players_names = [i.split('/')[5] for i in links]
players_names = [i.replace('-',' ') for i in players_names]

In [11]:
#Récupération du nom des attributs qui sont les mêmes pour chaque joueur

r = requests.get(links[0])
soup = BeautifulSoup(r.content, 'html.parser')
attrs = soup.select('div.block-quarter ul.pl li span')[7:75:2]
attrs_name = [i.text for i in attrs]

In [13]:
attrs_name[:5]

['Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys']

## Scrapping Players Infos

In [ ]:
%%time

players_ratings = []
compteur = 0

for p in links:
    r = requests.get(p)
    soup = BeautifulSoup(r.content, 'html.parser') 
      
    
    attrib = soup.select('div.block-quarter ul.pl li span.bp3-tag.p')[1:]
    ratings = [int(i.text) for i in attrib]
    
    
    
    compteur+= 1
    if compteur % 1000 == 0:
        
        print(f"Nombre de joueurs scrapés: {compteur}")
    players_ratings.append(ratings)


Nombre de joueurs scrapés: 1000
Nombre de joueurs scrapés: 2000
Nombre de joueurs scrapés: 3000


In [90]:
df = pd.DataFrame(data = players_ratings)
df.drop(columns = '34', inplace=True)
df.columns = attrs_name
df.insert(0, 'Player_Name', players_names)

In [ ]:
%%time

overall_ratings = []
potentials = []
market_value = []
best_positions = []
salaries = []
ages = []

compteur = 0

for p in links:
    r = requests.get(p)
    soup = BeautifulSoup(r.content, 'html.parser') 
    
    ov_ratings = soup.select('div.block-quarter')[0].text
    age = soup.select('div.meta.ellipsis')[0].text
    potential = soup.select('div.block-quarter span.bp3-tag.p')[1].text
    value = soup.select('div.block-quarter')[2].select('div')[0].text
    position = soup.select('li.ellipsis span.pos')[0].text
    salary = soup.select('div.block-quarter')[3].select('div')[0].text
    
    overall_ratings.append(ov_ratings)
    ages.append(age)
    potentials.append(potential)
    market_value.append(value)
    best_positions.append(position)
    salaries.append(salary)
    
    compteur+=1
    
    if compteur % 1000 == 0:
        print(f'{compteur} joueurs scrapés')

In [ ]:
df.insert(1,'Age',ages)
df.insert(2,'Position',best_positions)
df.insert(3,'Market_Value',market_value)
df.insert(4,'Salary',salaries)
df.insert(5,'Overall_ratings', overall_ratings)
df.insert(6,'Potential',potentials)

In [ ]:
df.to_csv('players_data.csv')